In [9]:
import math
import numpy as np
import scipy
import scipy.stats as ss
import sklearn.cluster as skc
import matplotlib.pyplot as plt
import pandas
import time
from time import mktime
from datetime import datetime

#suppresses pandas indexing warning for cleaner output
#don't worry, I know what I'm doing :)
pandas.options.mode.chained_assignment = None  # default='warn'

admissions = pandas.read_csv('data/ADMISSIONS.csv')
patients = pandas.read_csv('data/PATIENTS.csv')
diagnoses_icd = pandas.read_csv('data/DIAGNOSES_ICD.csv')
d_icd_diagnoses = pandas.read_csv('data/D_ICD_DIAGNOSES.csv')
services = pandas.read_csv('data/SERVICES.csv')

In [10]:
#concatenate data
data = pandas.merge(admissions,patients,on='SUBJECT_ID',how='left')
data = pandas.merge(data,services,on='HADM_ID',how='left')
data.drop('SUBJECT_ID_y', axis=1, inplace=True)
data.drop('PREV_SERVICE', axis=1, inplace=True)
data.drop('ROW_ID_x', axis=1, inplace=True)
data.rename(columns={'SUBJECT_ID_x': 'SUBJECT_ID'}, inplace=True)

In [11]:
#parses date input of the form "YYYY-MM-DD HH:MM:SS"
#converts it to a datetime
def parse_time(time_str):
    if (pandas.notnull(time_str)):
        temp_time = time.strptime(time_str, "%Y-%m-%d %H:%M:%S")
        return datetime.fromtimestamp(mktime(temp_time))
    else:
        return time_str
    
def sec_between(x,y):
    return (parse_time(y) - parse_time(x)).total_seconds()

def calc_staylength(row):
    return sec_between(row['ADMITTIME'],row['DISCHTIME'])

def calc_age(row):
    return sec_between(row['DOB'],row['ADMITTIME'])

In [12]:
data.reset_index(drop=True,inplace=True)
data['STAYLENGTH'] = data.apply(lambda row: calc_staylength(row),axis=1)
data['GENDER'] = data['GENDER'].map({'F':0,'M':1})

data['NEXTVISIT']=np.nan

sec_in_day = (60*60*24)
large_offset = 100000

for i in range(len(data)-1):
    if(data.ix[i,'SUBJECT_ID'] == data.ix[i+1,'SUBJECT_ID']):
        data.ix[i,'NEXTVISIT'] = sec_between(data.ix[i,'ADMITTIME'],data.ix[i+1,'ADMITTIME'])
    else:
        data.ix[i,'NEXTVISIT'] = sec_in_day * large_offset
data.ix[len(data)-1,'NEXTVISIT'] = sec_in_day * large_offset

data['STAYLENGTH'] = data['STAYLENGTH'] / sec_in_day
data['NEXTVISIT'] = data['NEXTVISIT'] / sec_in_day

data = data[data['NEXTVISIT'] != 0]

In [13]:
data_sparse = data.copy()
unique_icd9 = diagnoses_icd['ICD9_CODE'].unique()

for code in unique_icd9:
    data_sparse[code] = 0

,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,...,59001,3862,5644,9152,2982,9994,E9399,E0070,6940,20930
0,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,...,0,0,0,0,0,0,0,0,0,0
1,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,...,0,0,0,0,0,0,0,0,0,0
2,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,...,0,0,0,0,0,0,0,0,0,0
3,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,...,0,0,0,0,0,0,0,0,0,0
4,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,...,0,0,0,0,0,0,0,0,0,0
5,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Medicare,NaN,...,0,0,0,0,0,0,0,0,0,0
6,27,134931,2191-11-30 22:16:00,2191-12-03 14:45:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Private,NaN,...,0,0,0,0,0,0,0,0,0,0
7,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,...,0,0,0,0,0,0,0,0,0,0
8,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,NaN,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,NaN,...,0,0,0,0,0,0,0,0,0,0
10,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for i in range(data_sparse.shape[0]):
    hadm_id = data_sparse['HADM_ID'].iloc(i)
    hadm_icd9_array = diagnoses_icd[diagnoses_icd['HADM_ID'] == hadm_id]
    for code in hadm_icd9_array['ICD9_CODE'].unique():
        data_sparse[code].iloc[i] = 1
        
data_sparse

In [14]:
data.to_csv('concatenated_data_sparse.csv', encoding='utf-8')